# Fluxo de Trabalho com Intervenção Humana usando o Microsoft Agent Framework

## 🎯 Objetivos de Aprendizagem

Neste notebook, você aprenderá como implementar fluxos de trabalho com **intervenção humana** utilizando o `RequestInfoExecutor` do Microsoft Agent Framework. Esse padrão poderoso permite pausar fluxos de trabalho de IA para obter entrada humana, tornando seus agentes interativos e dando aos humanos controle sobre decisões críticas.

## 🔄 O que é Intervenção Humana?

**Intervenção humana (HITL)** é um padrão de design onde agentes de IA pausam a execução para solicitar entrada humana antes de continuar. Isso é essencial para:

- ✅ **Decisões críticas** - Obter aprovação humana antes de tomar ações importantes
- ✅ **Situações ambíguas** - Permitir que humanos esclareçam quando a IA está incerta
- ✅ **Preferências do usuário** - Perguntar aos usuários para escolher entre várias opções
- ✅ **Conformidade e segurança** - Garantir supervisão humana em operações regulamentadas
- ✅ **Experiências interativas** - Criar agentes conversacionais que respondem à entrada do usuário

## 🏗️ Como Funciona no Microsoft Agent Framework

O framework fornece três componentes principais para HITL:

1. **`RequestInfoExecutor`** - Um executor especial que pausa o fluxo de trabalho e emite um `RequestInfoEvent`
2. **`RequestInfoMessage`** - Classe base para cargas úteis de solicitação tipadas enviadas aos humanos
3. **`RequestResponse`** - Correlaciona respostas humanas com solicitações originais usando `request_id`

**Padrão de Fluxo de Trabalho:**
```
Agent detects need for input
    ↓
Sends message to RequestInfoExecutor
    ↓
Workflow pauses & emits RequestInfoEvent
    ↓
Application collects human input (console, UI, etc.)
    ↓
Application sends RequestResponse via send_responses_streaming()
    ↓
Workflow resumes with human input
```

## 🏨 Nosso Exemplo: Reserva de Hotel com Confirmação do Usuário

Vamos expandir o fluxo de trabalho condicional adicionando uma confirmação humana **antes** de sugerir destinos alternativos:

1. O usuário solicita um destino (por exemplo, "Paris")
2. O `availability_agent` verifica se há quartos disponíveis
3. **Se não houver quartos** → O `confirmation_agent` pergunta "Gostaria de ver alternativas?"
4. O fluxo de trabalho **pausa** usando o `RequestInfoExecutor`
5. **Humano responde** "sim" ou "não" via entrada no console
6. O `decision_manager` direciona com base na resposta:
   - **Sim** → Mostrar destinos alternativos
   - **Não** → Cancelar solicitação de reserva
7. Exibir resultado final

Isso demonstra como dar aos usuários controle sobre as sugestões do agente!

---

Vamos começar! 🚀


## Etapa 1: Importar Bibliotecas Necessárias

Importamos os componentes padrão do Agent Framework, além das **classes específicas para interação humana no processo**:
- `RequestInfoExecutor` - Executor que pausa o fluxo de trabalho para entrada humana
- `RequestInfoEvent` - Evento emitido quando é solicitada uma entrada humana
- `RequestInfoMessage` - Classe base para cargas úteis de solicitação tipadas
- `RequestResponse` - Correlaciona respostas humanas com solicitações
- `WorkflowOutputEvent` - Evento para detectar saídas do fluxo de trabalho


In [21]:
import asyncio
import json
import os
from dataclasses import dataclass
from typing import Annotated, Any, Never

from agent_framework import (
    AgentExecutor,
    AgentExecutorRequest,
    AgentExecutorResponse,
    ChatMessage,
    Executor,
    RequestInfoEvent,          # NEW: Event when human input is requested
    RequestInfoExecutor,       # NEW: Executor that gathers human input
    RequestInfoMessage,        # NEW: Base class for request payloads
    RequestResponse,           # NEW: Correlates response with request
    Role,
    WorkflowBuilder,
    WorkflowContext,
    WorkflowOutputEvent,       # NEW: Event for workflow outputs
    WorkflowRunState,          # NEW: Enum of workflow run states
    WorkflowStatusEvent,       # NEW: Event for run state changes
    ai_function,
    executor,
    handler,                   # NEW: Decorator for executor methods
)

# 🤖 GitHub Models or OpenAI client integration
from agent_framework.openai import OpenAIChatClient
from dotenv import load_dotenv
from IPython.display import HTML, display
from pydantic import BaseModel

print("✅ All imports successful!")
print("🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse")

✅ All imports successful!
🔄 Human-in-the-loop components loaded: RequestInfoExecutor, RequestInfoEvent, RequestResponse


## Etapa 2: Definir Modelos Pydantic para Saídas Estruturadas

Esses modelos definem o **esquema** que os agentes irão retornar. Mantemos todos os modelos do fluxo de trabalho condicional e adicionamos:

**Novidade para Intervenção Humana:**
- `HumanFeedbackRequest` - Subclasse de `RequestInfoMessage` que define o payload da solicitação enviado para humanos
  - Contém `prompt` (pergunta a ser feita) e `destination` (contexto sobre a cidade indisponível)


In [22]:
# Existing models from conditional workflow
class BookingCheckResult(BaseModel):
    """Result from checking hotel availability at a destination."""
    destination: str
    has_availability: bool
    message: str


class AlternativeResult(BaseModel):
    """Suggested alternative destination when no rooms available."""
    alternative_destination: str
    reason: str


class BookingConfirmation(BaseModel):
    """Booking suggestion when rooms are available."""
    destination: str
    action: str
    message: str


# NEW: Pydantic model for agent's response format
class ConfirmationQuestion(BaseModel):
    """
    Pydantic model used by confirmation_agent's response_format.
    This is what the agent will output as JSON.
    """
    question: str  # The question to ask the user
    destination: str  # The unavailable destination for context


# NEW: Dataclass for RequestInfoExecutor
@dataclass
class HumanFeedbackRequest(RequestInfoMessage):
    """
    Request sent to RequestInfoExecutor asking if user wants alternatives.
    
    MUST be a dataclass subclassing RequestInfoMessage for type compatibility.
    This is what gets sent to the RequestInfoExecutor.
    """
    prompt: str = ""  # The question to ask the user
    destination: str = ""  # The unavailable destination for context


print("✅ Pydantic models defined:")
print("   - BookingCheckResult (availability check)")
print("   - AlternativeResult (alternative suggestion)")
print("   - BookingConfirmation (booking confirmation)")
print("   - ConfirmationQuestion (agent response format) 🆕")
print("   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕")

✅ Pydantic models defined:
   - BookingCheckResult (availability check)
   - AlternativeResult (alternative suggestion)
   - BookingConfirmation (booking confirmation)
   - ConfirmationQuestion (agent response format) 🆕
   - HumanFeedbackRequest (RequestInfoMessage for HITL) 🆕


## Etapa 3: Criar a Ferramenta de Reserva de Hotel

Mesma ferramenta do fluxo de trabalho condicional - verifica se há quartos disponíveis no destino.


In [23]:
@ai_function(description="Check hotel room availability for a destination city")
def hotel_booking(destination: Annotated[str, "The destination city to check for hotel rooms"]) -> str:
    """
    Simulates checking hotel room availability.
    
    Returns JSON string with availability status.
    """
    display(
        HTML(f"""
        <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
            <strong>🔍 Tool Invoked:</strong> hotel_booking("{destination}")
        </div>
    """)
    )

    # Simulate availability check
    cities_with_rooms = ["stockholm", "seattle", "tokyo", "london", "amsterdam"]
    has_rooms = destination.lower() in cities_with_rooms

    result = {"has_availability": has_rooms, "destination": destination}

    return json.dumps(result)


print("✅ hotel_booking tool created with @ai_function decorator")

✅ hotel_booking tool created with @ai_function decorator


## Etapa 4: Definir Funções de Condição para Roteamento

Precisamos de **quatro funções de condição** para nosso fluxo de trabalho com intervenção humana:

**Do fluxo de trabalho condicional:**
1. `has_availability_condition` - Roteia quando hotéis ESTÃO disponíveis
2. `no_availability_condition` - Roteia quando hotéis NÃO estão disponíveis

**Novas para intervenção humana:**
3. `user_wants_alternatives_condition` - Roteia quando o usuário diz "sim" para alternativas
4. `user_declines_alternatives_condition` - Roteia quando o usuário diz "não" para alternativas


In [24]:
# Existing condition functions from conditional workflow
def has_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels ARE available."""
    if not isinstance(message, AgentExecutorResponse):
        return True

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                <strong>✅ Condition Check:</strong> has_availability = <strong>{result.has_availability}</strong> for {result.destination}
            </div>
        """)
        )
        return result.has_availability
    except Exception as e:
        display(HTML(f"""<div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'><strong>⚠️  Error:</strong> {str(e)}</div>"""))
        return False


def no_availability_condition(message: Any) -> bool:
    """Condition for routing when hotels are NOT available."""
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        result = BookingCheckResult.model_validate_json(message.agent_run_response.text)
        display(
            HTML(f"""
            <div style='padding: 12px; background: #ffecb3; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                <strong>❌ Condition Check:</strong> no_availability for {result.destination}
            </div>
        """)
        )
        return not result.has_availability
    except Exception as e:
        return False


# NEW: Condition functions for human-in-the-loop routing
def user_wants_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user WANTS to see alternatives.
    
    Checks the AgentExecutorResponse from decision_manager to see if user said 'yes'.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        # The decision_manager yields a simple text response
        response_text = message.agent_run_response.text.lower().strip()
        
        display(
            HTML(f"""
            <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
                <strong>🔍 User Decision:</strong> User wants alternatives = <strong>{response_text == 'yes'}</strong>
            </div>
        """)
        )
        
        return response_text == "yes"
    except Exception as e:
        return False


def user_declines_alternatives_condition(message: Any) -> bool:
    """
    Condition for routing when user DECLINES alternatives.
    
    Checks the AgentExecutorResponse from decision_manager to see if user said 'no'.
    """
    if not isinstance(message, AgentExecutorResponse):
        return False

    try:
        response_text = message.agent_run_response.text.lower().strip()
        
        display(
            HTML(f"""
            <div style='padding: 12px; background: #fce4ec; border-left: 4px solid #c2185b; border-radius: 4px; margin: 10px 0;'>
                <strong>🚫 User Decision:</strong> User declined alternatives = <strong>{response_text == 'no'}</strong>
            </div>
        """)
        )
        
        return response_text == "no"
    except Exception as e:
        return False


print("✅ Condition functions defined:")
print("   - has_availability_condition (routes when rooms exist)")
print("   - no_availability_condition (routes when no rooms)")
print("   - user_wants_alternatives_condition (routes when user says yes) 🆕")
print("   - user_declines_alternatives_condition (routes when user says no) 🆕")

✅ Condition functions defined:
   - has_availability_condition (routes when rooms exist)
   - no_availability_condition (routes when no rooms)
   - user_wants_alternatives_condition (routes when user says yes) 🆕
   - user_declines_alternatives_condition (routes when user says no) 🆕


## Etapa 5: Criar o Executor do Gerenciador de Decisões

Este é o **núcleo do padrão human-in-the-loop**! O `DecisionManager` é um `Executor` personalizado que:

1. **Recebe feedback humano** por meio de objetos `RequestResponse`
2. **Processa a decisão do usuário** (sim/não)
3. **Direciona o fluxo de trabalho** enviando mensagens para os agentes apropriados

Principais características:
- Utiliza o decorador `@handler` para expor métodos como etapas do fluxo de trabalho
- Recebe `RequestResponse[HumanFeedbackRequest, str]` contendo tanto a solicitação original quanto a resposta do usuário
- Gera mensagens simples de "sim" ou "não" que acionam nossas funções de condição


In [25]:
class DecisionManager(Executor):
    """
    Coordinates workflow routing based on human feedback.
    
    This executor receives RequestResponse objects from the RequestInfoExecutor
    and makes routing decisions by sending simple messages that trigger
    condition functions.
    """

    def __init__(self, id: str | None = None):
        super().__init__(id=id or "decision_manager")

    @handler
    async def on_human_feedback(
        self,
        feedback: RequestResponse[HumanFeedbackRequest, str],
        ctx: WorkflowContext[AgentExecutorRequest],
    ) -> None:
        """
        Process human feedback and route workflow accordingly.
        
        The RequestResponse contains:
        - feedback.data: The user's string reply (e.g., "yes" or "no")
        - feedback.original_request: The HumanFeedbackRequest with context
        
        We send a simple message that will be caught by our condition functions.
        """
        user_reply = (feedback.data or "").strip().lower()
        destination = getattr(feedback.original_request, "destination", "unknown")

        display(
            HTML(f"""
            <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
                <strong>🎯 Decision Manager:</strong> Processing user reply: <strong>"{user_reply}"</strong> for {destination}
            </div>
        """)
        )

        if user_reply == "yes":
            # User wants alternatives - send message to trigger alternative_agent
            display(
                HTML("""
                <div style='padding: 12px; background: #c8e6c9; border-left: 4px solid #4caf50; border-radius: 4px; margin: 10px 0;'>
                    <strong>➡️  Routing:</strong> User wants alternatives → Sending to alternative_agent
                </div>
            """)
            )
            # Send a message requesting alternatives for the destination
            user_msg = ChatMessage(
                Role.USER,
                text=f"The user wants to see alternative destinations near {destination}. Please suggest one.",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        elif user_reply == "no":
            # User declined - send message to trigger cancellation
            display(
                HTML("""
                <div style='padding: 12px; background: #ffcdd2; border-left: 4px solid #f44336; border-radius: 4px; margin: 10px 0;'>
                    <strong>🚫 Routing:</strong> User declined alternatives → Sending cancellation message
                </div>
            """)
            )
            # Send a simple "no" message that will be caught by the condition
            user_msg = ChatMessage(
                Role.USER,
                text="no",
            )
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))
        
        else:
            # Handle unexpected input
            display(
                HTML(f"""
                <div style='padding: 12px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 4px; margin: 10px 0;'>
                    <strong>⚠️  Warning:</strong> Unexpected input "{user_reply}" - treating as decline
                </div>
            """)
            )
            user_msg = ChatMessage(Role.USER, text="no")
            await ctx.send_message(AgentExecutorRequest(messages=[user_msg], should_respond=True))


print("✅ DecisionManager executor created with @handler method for human feedback")

✅ DecisionManager executor created with @handler method for human feedback


## Etapa 6: Criar Executor de Exibição Personalizado

Mesmo executor de exibição do fluxo de trabalho condicional - gera os resultados finais como saída do fluxo de trabalho.


In [26]:
@executor(id="prepare_human_request")
async def prepare_human_request(
    response: AgentExecutorResponse, 
    ctx: WorkflowContext[HumanFeedbackRequest]
) -> None:
    """
    Transform agent response into HumanFeedbackRequest for RequestInfoExecutor.
    
    This executor bridges the type gap between:
    - confirmation_agent outputs AgentExecutorResponse with ConfirmationQuestion JSON
    - request_info_executor expects HumanFeedbackRequest (RequestInfoMessage dataclass)
    """
    display(
        HTML("""
        <div style='padding: 12px; background: #e1f5fe; border-left: 4px solid #0288d1; border-radius: 4px; margin: 10px 0;'>
            <strong>🔄 Transform:</strong> Converting ConfirmationQuestion to HumanFeedbackRequest
        </div>
    """)
    )
    
    # Parse the agent's Pydantic output (ConfirmationQuestion)
    confirmation = ConfirmationQuestion.model_validate_json(response.agent_run_response.text)
    
    # Convert to HumanFeedbackRequest dataclass for RequestInfoExecutor
    feedback_request = HumanFeedbackRequest(
        prompt=confirmation.question,
        destination=confirmation.destination
    )
    
    # Send the properly typed RequestInfoMessage to the RequestInfoExecutor
    await ctx.send_message(feedback_request)


@executor(id="display_result")
async def display_result(response: AgentExecutorResponse, ctx: WorkflowContext[Never, str]) -> None:
    """
    Display the final result as workflow output.
    
    This executor receives the final agent response and yields it as the workflow output.
    """
    display(
        HTML("""
        <div style='padding: 15px; background: #f3e5f5; border-left: 4px solid #9c27b0; border-radius: 4px; margin: 10px 0;'>
            <strong>📤 Display Executor:</strong> Yielding workflow output
        </div>
    """)
    )

    await ctx.yield_output(response.agent_run_response.text)


print("✅ prepare_human_request executor created with @executor decorator")
print("✅ display_result executor created with @executor decorator")

✅ prepare_human_request executor created with @executor decorator
✅ display_result executor created with @executor decorator


## Etapa 7: Carregar Variáveis de Ambiente

Configure o cliente LLM (Modelos do GitHub, Azure OpenAI ou OpenAI).


In [27]:
# Load environment variables
load_dotenv()

# Check for GitHub Models or OpenAI
chat_client = OpenAIChatClient(
    base_url=os.environ.get("GITHUB_ENDPOINT"), 
    api_key=os.environ.get("GITHUB_TOKEN"), 
    model_id="gpt-4o"
)

print("✅ Chat client configured with GitHub Models")

✅ Chat client configured with GitHub Models


## Etapa 8: Criar Agentes de IA e Executores

Criamos **seis componentes de fluxo de trabalho**:

**Agentes (envolvidos em AgentExecutor):**
1. **availability_agent** - Verifica a disponibilidade de hotéis usando a ferramenta
2. **confirmation_agent** - 🆕 Prepara a solicitação de confirmação humana
3. **alternative_agent** - Sugere cidades alternativas (quando o usuário diz sim)
4. **booking_agent** - Incentiva a reserva (quando há quartos disponíveis)
5. **cancellation_agent** - 🆕 Lida com a mensagem de cancelamento (quando o usuário diz não)

**Executores Especiais:**
6. **request_info_executor** - 🆕 `RequestInfoExecutor` que pausa o fluxo de trabalho para entrada humana
7. **decision_manager** - 🆕 Executor personalizado que direciona com base na resposta humana (já definido acima)


In [28]:
# Agent 1: Check availability with tool (same as conditional workflow)
availability_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a hotel booking assistant that checks room availability. "
            "Use the hotel_booking tool to check if rooms are available at the destination. "
            "Return JSON with fields: destination (string), has_availability (bool), and message (string). "
            "The message should summarize the availability status."
        ),
        tools=[hotel_booking],
        response_format=BookingCheckResult,
    ),
    id="availability_agent",
)

# Agent 2: NEW - Prepare human confirmation request
confirmation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user's requested destination has no available hotel rooms. "
            "Create a polite message asking if they would like to see alternative destinations nearby. "
            "Return a JSON with: destination (the unavailable city), and question (a friendly yes/no question). "
            "Keep the question concise and friendly."
        ),
        response_format=ConfirmationQuestion,  # Use Pydantic model for agent output
    ),
    id="confirmation_agent",
)

# Agent 3: Suggest alternative (when user says yes)
alternative_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful travel assistant. When a user cannot find hotels in their requested city, "
            "suggest an alternative nearby city that has availability. "
            "Return JSON with fields: alternative_destination (string) and reason (string). "
            "Make your suggestion sound appealing and helpful."
        ),
        response_format=AlternativeResult,
    ),
    id="alternative_agent",
)

# Agent 4: Suggest booking (when rooms available)
booking_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a booking assistant. The user has found available hotel rooms. "
            "Encourage them to book by highlighting the destination's appeal. "
            "Return JSON with fields: destination (string), action (string), and message (string). "
            "The action should be 'book_now' and message should be encouraging."
        ),
        response_format=BookingConfirmation,
    ),
    id="booking_agent",
)

# Agent 5: NEW - Handle cancellation when user declines alternatives
class CancellationMessage(BaseModel):
    """Message when user declines alternatives."""
    status: str
    message: str

cancellation_agent = AgentExecutor(
    chat_client.create_agent(
        instructions=(
            "You are a helpful assistant. The user has declined to see alternative hotel destinations. "
            "Create a polite cancellation message. "
            "Return JSON with: status (should be 'cancelled'), and message (a friendly acknowledgment). "
            "Keep the message brief and understanding."
        ),
        response_format=CancellationMessage,
    ),
    id="cancellation_agent",
)

# NEW: RequestInfoExecutor - pauses workflow to gather human input
request_info_executor = RequestInfoExecutor(id="request_info")

# NEW: DecisionManager instance - routes based on human feedback
decision_manager = DecisionManager(id="decision_manager")

display(
    HTML("""
    <div style='padding: 15px; background: #e3f2fd; border-left: 4px solid #2196f3; border-radius: 4px; margin: 10px 0;'>
        <strong>✅ Created Workflow Components:</strong>
        <ul style='margin: 10px 0 0 0;'>
            <li><strong>availability_agent</strong> - Checks availability with hotel_booking tool</li>
            <li><strong>confirmation_agent</strong> 🆕 - Prepares human confirmation request</li>
            <li><strong>alternative_agent</strong> - Suggests alternative cities</li>
            <li><strong>booking_agent</strong> - Encourages booking</li>
            <li><strong>cancellation_agent</strong> 🆕 - Handles user declining alternatives</li>
            <li><strong>request_info_executor</strong> 🆕 - Pauses workflow for human input</li>
            <li><strong>decision_manager</strong> 🆕 - Routes based on human response</li>
        </ul>
    </div>
""")
)

## Etapa 9: Construir o Fluxo de Trabalho com Intervenção Humana

Agora vamos construir o gráfico do fluxo de trabalho com **roteamento condicional**, incluindo o caminho com intervenção humana:

**Estrutura do Fluxo de Trabalho:**
```
availability_agent (START)
        ↓
   Evaluate conditions
        ↙                    ↘
[no_availability]        [has_availability]
        ↓                        ↓
confirmation_agent          booking_agent
        ↓                        ↓
prepare_human_request      display_result
        ↓
request_info_executor (PAUSE)
        ↓
decision_manager
   ↙         ↘
[yes]        [no]
   ↓           ↓
alternative  cancellation
   ↓           ↓
display_result
```

**Conexões Principais:**
- `availability_agent → confirmation_agent` (quando não há quartos disponíveis)
- `confirmation_agent → prepare_human_request` (transformar tipo)
- `prepare_human_request → request_info_executor` (pausa para intervenção humana)
- `request_info_executor → decision_manager` (sempre - fornece RequestResponse)
- `decision_manager → alternative_agent` (quando o usuário diz "sim")
- `decision_manager → cancellation_agent` (quando o usuário diz "não")
- `availability_agent → booking_agent` (quando há quartos disponíveis)
- Todos os caminhos terminam em `display_result`


In [29]:
# Build the workflow with human-in-the-loop routing
workflow = (
    WorkflowBuilder()
    .set_start_executor(availability_agent)
    
    # NO AVAILABILITY PATH (with human-in-the-loop)
    .add_edge(availability_agent, confirmation_agent, condition=no_availability_condition)
    .add_edge(confirmation_agent, prepare_human_request)  # Transform to HumanFeedbackRequest
    .add_edge(prepare_human_request, request_info_executor)  # Send to RequestInfoExecutor
    .add_edge(request_info_executor, decision_manager)    # Always goes to decision manager
    
    # Decision manager routes based on user response
    .add_edge(decision_manager, alternative_agent, condition=user_wants_alternatives_condition)
    .add_edge(decision_manager, cancellation_agent, condition=user_declines_alternatives_condition)
    .add_edge(alternative_agent, display_result)
    .add_edge(cancellation_agent, display_result)
    
    # HAS AVAILABILITY PATH (no human input needed)
    .add_edge(availability_agent, booking_agent, condition=has_availability_condition)
    .add_edge(booking_agent, display_result)
    
    .build()
)

display(
    HTML("""
    <div style='padding: 20px; background: linear-gradient(135deg, #667eea 0%, #764ba2 100%); color: white; border-radius: 8px; margin: 10px 0;'>
        <h3 style='margin: 0 0 15px 0;'>✅ Workflow Built Successfully!</h3>
        <p style='margin: 0; line-height: 1.6;'>
            <strong>Human-in-the-Loop Routing:</strong><br>
            • If <strong>NO availability</strong> → confirmation_agent → prepare_human_request → request_info_executor → <strong>PAUSE FOR HUMAN</strong> → decision_manager<br>
            &nbsp;&nbsp;• If user says <strong>YES</strong> → alternative_agent → display_result<br>
            &nbsp;&nbsp;• If user says <strong>NO</strong> → cancellation_agent → display_result<br>
            • If <strong>availability</strong> → booking_agent → display_result (no human input needed)
        </p>
    </div>
""")
)

## Etapa 10: Executar Caso de Teste 1 - Cidade SEM Disponibilidade (Paris com Confirmação Humana)

Este teste demonstra o **ciclo completo com intervenção humana**:

1. Solicitar hotel em Paris
2. availability_agent verifica → Sem quartos disponíveis
3. confirmation_agent cria uma pergunta para o usuário
4. request_info_executor **pausa o fluxo de trabalho** e emite `RequestInfoEvent`
5. **Aplicativo detecta o evento e solicita ao usuário no console**
6. Usuário digita "sim" ou "não"
7. Aplicativo envia a resposta via `send_responses_streaming()`
8. decision_manager direciona com base na resposta
9. Resultado final exibido

**Padrão Principal:**
- Use `workflow.run_stream()` para a primeira iteração
- Use `workflow.send_responses_streaming(pending_responses)` para as iterações subsequentes
- Escute por `RequestInfoEvent` para detectar quando é necessário input humano
- Escute por `WorkflowOutputEvent` para capturar os resultados finais


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #fff3e0; border-left: 4px solid #ff9800; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #e65100;'>🧪 TEST CASE 1: Paris (No Availability - Human-in-the-Loop)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → confirmation_agent → request_info_executor → <strong>PAUSE</strong> → decision_manager → (depends on user input)</p>
    </div>
""")
)

# Create request for Paris
request_paris = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Paris")], 
    should_respond=True
)

# Human-in-the-loop execution pattern
pending_responses: dict[str, str] | None = None
completed = False
workflow_output: str | None = None

print("\n🔄 Starting human-in-the-loop workflow...")
print("=" * 60)

while not completed:
    # First iteration uses run_stream with the request
    # Subsequent iterations use send_responses_streaming with collected human responses
    if pending_responses:
        print(f"\n📤 Sending human responses: {pending_responses}")
        stream = workflow.send_responses_streaming(pending_responses)
        pending_responses = None  # Clear immediately after sending
    else:
        print(f"\n🚀 Starting workflow with request: 'I want to book a hotel in Paris'")
        stream = workflow.run_stream(request_paris)
    
    # Collect all events from this iteration
    events = [event async for event in stream]
    
    # Process events
    requests: list[tuple[str, str]] = []  # (request_id, prompt)
    
    for event in events:
        # Check for human input requests
        if isinstance(event, RequestInfoEvent) and isinstance(event.data, HumanFeedbackRequest):
            print(f"\n⏸️  WORKFLOW PAUSED - Human input requested!")
            print(f"   Request ID: {event.request_id}")
            print(f"   Destination: {event.data.destination}")
            requests.append((event.request_id, event.data.prompt))
        
        # Check for workflow outputs
        elif isinstance(event, WorkflowOutputEvent):
            workflow_output = str(event.data)
            completed = True
            print(f"\n✅ Workflow completed with output!")
    
    # If we have human requests, prompt the user
    if requests and not completed:
        responses: dict[str, str] = {}
        for req_id, prompt in requests:
            print(f"\n{'='*60}")
            print(f"💬 QUESTION FOR YOU:")
            print(f"   {prompt}")
            print(f"{'='*60}")
            
            # Get user input (in notebook, this will pause execution)
            answer = input("👉 Enter 'yes' or 'no': ").strip().lower()
            
            print(f"\n📝 You answered: {answer}")
            responses[req_id] = answer
        
        pending_responses = responses

print(f"\n{'='*60}")
print(f"🏆 FINAL WORKFLOW OUTPUT:")
print(f"{'='*60}")

# Display final result
if workflow_output:
    # Try to parse as JSON for pretty display
    try:
        result_data = json.loads(workflow_output)
        if "alternative_destination" in result_data:
            result_obj = AlternativeResult.model_validate_json(workflow_output)
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #FFD700 0%, #FFA500 100%); border-radius: 12px; box-shadow: 0 4px 12px rgba(255,165,0,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0; color: #333;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> ✅ Accepted alternatives</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Alternative Suggestion:</strong> 🏨 {result_obj.alternative_destination}</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Reason:</strong> {result_obj.reason}</p>
                    </div>
                </div>
            """)
            )
        else:
            # User declined
            display(
                HTML(f"""
                <div style='padding: 25px; background: linear-gradient(135deg, #f44336 0%, #e91e63 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(244,67,54,0.3); margin: 20px 0;'>
                    <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT</h3>
                    <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ❌ No rooms in Paris</p>
                        <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>User Decision:</strong> 🚫 Declined alternatives</p>
                        <p style='margin: 0; font-size: 14px; color: #666;'><strong>Result:</strong> Booking request cancelled</p>
                    </div>
                </div>
            """)
            )
    except:
        print(workflow_output)


🔄 Starting human-in-the-loop workflow...

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: 032c8fce-b9d1-400e-ba8d-afd2248e2926
   Destination: Paris

💬 QUESTION FOR YOU:
   Unfortunately, there are no rooms available in Paris. Would you like to explore nearby alternative destinations?

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: yes

📤 Sending human responses: {'032c8fce-b9d1-400e-ba8d-afd2248e2926': 'yes'}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'



⏸️  WORKFLOW PAUSED - Human input requested!
   Request ID: cf48dad0-ee5e-4f60-8806-341a7a292bd4
   Destination: Paris

💬 QUESTION FOR YOU:
   I'm sorry to inform you that there are no available hotel rooms in Paris. Would you like me to suggest nearby alternative destinations?

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'

📝 You answered: 

📤 Sending human responses: {'cf48dad0-ee5e-4f60-8806-341a7a292bd4': ''}

🚀 Starting workflow with request: 'I want to book a hotel in Paris'


## Etapa 11: Executar Caso de Teste 2 - Cidade COM Disponibilidade (Estocolmo - Sem Necessidade de Interação Humana)

Este teste demonstra o **caminho direto** quando há quartos disponíveis:

1. Solicitar hotel em Estocolmo
2. availability_agent verifica → Quartos disponíveis ✅
3. booking_agent sugere a reserva
4. display_result mostra a confirmação
5. **Nenhuma interação humana necessária!**

O fluxo ignora completamente o caminho com intervenção humana quando há quartos disponíveis.


In [ ]:
display(
    HTML("""
    <div style='padding: 20px; background: #e8f5e9; border-left: 4px solid #4caf50; border-radius: 8px; margin: 20px 0;'>
        <h3 style='margin: 0 0 10px 0; color: #1b5e20;'>🧪 TEST CASE 2: Stockholm (Has Availability - No Human Input)</h3>
        <p style='margin: 0;'>Expected workflow path: availability_agent → booking_agent → display_result (direct, no pause)</p>
    </div>
""")
)

# Create request for Stockholm
request_stockholm = AgentExecutorRequest(
    messages=[ChatMessage(Role.USER, text="I want to book a hotel in Stockholm")], 
    should_respond=True
)

# Run the workflow (should complete without human input)
events_stockholm = await workflow.run(request_stockholm)
outputs_stockholm = events_stockholm.get_outputs()

# Display results
if outputs_stockholm:
    result_stockholm = BookingConfirmation.model_validate_json(outputs_stockholm[0])

    display(
        HTML(f"""
        <div style='padding: 25px; background: linear-gradient(135deg, #4caf50 0%, #8bc34a 100%); color: white; border-radius: 12px; box-shadow: 0 4px 12px rgba(76,175,80,0.3); margin: 20px 0;'>
            <h3 style='margin: 0 0 15px 0;'>🏆 WORKFLOW RESULT (Stockholm - No Human Input)</h3>
            <div style='background: white; color: #333; padding: 20px; border-radius: 8px;'>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Status:</strong> ✅ Rooms Available!</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Destination:</strong> 🏨 {result_stockholm.destination}</p>
                <p style='margin: 0 0 10px 0; font-size: 16px;'><strong>Action:</strong> {result_stockholm.action}</p>
                <p style='margin: 0 0 10px 0; font-size: 14px; color: #666;'><strong>Message:</strong> {result_stockholm.message}</p>
                <p style='margin: 10px 0 0 0; font-size: 12px; color: #999; font-style: italic;'>Note: No human input was requested because rooms were available!</p>
            </div>
        </div>
    """)
    )

## Principais Aprendizados e Melhores Práticas para Human-in-the-Loop

### ✅ O que você aprendeu:

#### 1. **Padrão RequestInfoExecutor**
O padrão human-in-the-loop no Microsoft Agent Framework utiliza três componentes principais:
- `RequestInfoExecutor` - Pausa o fluxo de trabalho e emite eventos
- `RequestInfoMessage` - Classe base para cargas úteis de solicitação tipadas (subclasse esta!)
- `RequestResponse` - Correlaciona respostas humanas com solicitações originais

**Entendimento Crítico:**
- `RequestInfoExecutor` NÃO coleta entrada por conta própria - ele apenas pausa o fluxo de trabalho
- Seu código de aplicação deve escutar o `RequestInfoEvent` e coletar a entrada
- Você deve chamar `send_responses_streaming()` com um dicionário mapeando `request_id` para a resposta do usuário

#### 2. **Padrão de Execução em Streaming**
```python
# First iteration
stream = workflow.run_stream(initial_request)

# Subsequent iterations (after human input)
stream = workflow.send_responses_streaming(pending_responses)

# Always process events
events = [event async for event in stream]
```

#### 3. **Arquitetura Orientada a Eventos**
Escute eventos específicos para controlar o fluxo de trabalho:
- `RequestInfoEvent` - Entrada humana é necessária (fluxo de trabalho pausado)
- `WorkflowOutputEvent` - Resultado final está disponível (fluxo de trabalho concluído)
- `WorkflowStatusEvent` - Mudanças de estado (IN_PROGRESS, IDLE_WITH_PENDING_REQUESTS, etc.)

#### 4. **Executores Personalizados com @handler**
O `DecisionManager` demonstra como criar executores que:
- Usam o decorador `@handler` para expor métodos como etapas do fluxo de trabalho
- Recebem mensagens tipadas (por exemplo, `RequestResponse[HumanFeedbackRequest, str]`)
- Roteiam o fluxo de trabalho enviando mensagens para outros executores
- Acessam o contexto via `WorkflowContext`

#### 5. **Roteamento Condicional com Decisões Humanas**
Você pode criar funções condicionais que avaliam respostas humanas:
```python
def user_wants_alternatives_condition(message: Any) -> bool:
    response_text = message.agent_run_response.text.lower()
    return response_text == "yes"
```

### 🎯 Aplicações no Mundo Real:

1. **Fluxos de Aprovação**
   - Obter aprovação do gerente antes de processar relatórios de despesas
   - Exigir revisão humana antes de enviar e-mails automatizados
   - Confirmar transações de alto valor antes da execução

2. **Moderação de Conteúdo**
   - Marcar conteúdo questionável para revisão humana
   - Pedir aos moderadores que tomem decisões finais em casos limítrofes
   - Escalar para humanos quando a confiança da IA for baixa

3. **Atendimento ao Cliente**
   - Permitir que a IA lide automaticamente com perguntas rotineiras
   - Escalar questões complexas para agentes humanos
   - Perguntar ao cliente se ele deseja falar com um humano

4. **Processamento de Dados**
   - Pedir aos humanos para resolver entradas de dados ambíguas
   - Confirmar interpretações da IA de documentos pouco claros
   - Permitir que os usuários escolham entre múltiplas interpretações válidas

5. **Sistemas Críticos de Segurança**
   - Exigir confirmação humana antes de ações irreversíveis
   - Obter aprovação antes de acessar dados sensíveis
   - Confirmar decisões em indústrias regulamentadas (saúde, finanças)

6. **Agentes Interativos**
   - Construir bots conversacionais que fazem perguntas de acompanhamento
   - Criar assistentes que guiam os usuários por processos complexos
   - Projetar agentes que colaboram com humanos passo a passo

### 🔄 Comparação: Com vs Sem Human-in-the-Loop

| Recurso | Fluxo de Trabalho Condicional | Fluxo de Trabalho Human-in-the-Loop |
|---------|-------------------------------|-------------------------------------|
| **Execução** | Único `workflow.run()` | Loop com `run_stream()` + `send_responses_streaming()` |
| **Entrada do Usuário** | Nenhuma (totalmente automatizado) | Prompts interativos via `input()` ou UI |
| **Componentes** | Agentes + Executores | + RequestInfoExecutor + DecisionManager |
| **Eventos** | Apenas AgentExecutorResponse | RequestInfoEvent, WorkflowOutputEvent, etc. |
| **Pausas** | Sem pausas | Fluxo de trabalho pausa no RequestInfoExecutor |
| **Controle Humano** | Sem controle humano | Humanos tomam decisões-chave |
| **Caso de Uso** | Automação | Colaboração e supervisão |

### 🚀 Padrões Avançados:

#### Múltiplos Pontos de Decisão Humana
Você pode ter múltiplos nós `RequestInfoExecutor` no mesmo fluxo de trabalho:
```python
.add_edge(agent1, request_info_1)  # First human decision
.add_edge(decision_manager_1, agent2)
.add_edge(agent2, request_info_2)  # Second human decision
.add_edge(decision_manager_2, final_agent)
```

#### Tratamento de Timeout
Implemente timeouts para respostas humanas:
```python
import asyncio

try:
    answer = await asyncio.wait_for(
        asyncio.to_thread(input, "Enter yes/no: "),
        timeout=60.0
    )
except asyncio.TimeoutError:
    answer = "no"  # Default to safe option
```

#### Integração com UI Avançada
Em vez de usar `input()`, integre com UI web, Slack, Teams, etc.:
```python
if isinstance(event, RequestInfoEvent):
    # Send notification to user's preferred channel
    await slack_client.send_message(
        user_id=current_user,
        text=event.data.prompt,
        request_id=event.request_id
    )
```

#### Human-in-the-Loop Condicional
Solicite entrada humana apenas em situações específicas:
```python
def needs_human_approval_condition(message: Any) -> bool:
    # Only route to human if confidence is low or value is high
    if result.confidence < 0.7 or result.value > 10000:
        return True
    return False
```

### ⚠️ Melhores Práticas:

1. **Sempre Subclasse RequestInfoMessage**
   - Fornece segurança de tipo e validação
   - Permite contexto rico para renderização de UI
   - Esclarece a intenção de cada tipo de solicitação

2. **Use Prompts Descritivos**
   - Inclua contexto sobre o que você está pedindo
   - Explique as consequências de cada escolha
   - Mantenha as perguntas simples e claras

3. **Trate Entradas Inesperadas**
   - Valide as respostas dos usuários
   - Forneça padrões para entradas inválidas
   - Dê mensagens de erro claras

4. **Rastreie IDs de Solicitação**
   - Use a correlação entre request_id e respostas
   - Não tente gerenciar estado manualmente

5. **Projete para Não Bloquear**
   - Não bloqueie threads esperando por entrada
   - Use padrões assíncronos em todo o sistema
   - Suporte instâncias de fluxo de trabalho concorrentes

### 📚 Conceitos Relacionados:

- **Middleware de Agente** - Interceptar chamadas de agentes (notebook anterior)
- **Gerenciamento de Estado do Fluxo de Trabalho** - Persistir estado do fluxo de trabalho entre execuções
- **Colaboração Multi-Agente** - Combinar human-in-the-loop com equipes de agentes
- **Arquiteturas Orientadas a Eventos** - Construir sistemas reativos com eventos

---

### 🎓 Parabéns!

Você dominou fluxos de trabalho human-in-the-loop com o Microsoft Agent Framework! Agora você sabe como:
- ✅ Pausar fluxos de trabalho para coletar entrada humana
- ✅ Usar RequestInfoExecutor e RequestInfoMessage
- ✅ Lidar com execução em streaming com eventos
- ✅ Criar executores personalizados com @handler
- ✅ Rotear fluxos de trabalho com base em decisões humanas
- ✅ Construir agentes de IA interativos que colaboram com humanos

**Este é um padrão crítico para construir sistemas de IA confiáveis e controláveis!** 🚀



---

**Aviso Legal**:  
Este documento foi traduzido utilizando o serviço de tradução por IA [Co-op Translator](https://github.com/Azure/co-op-translator). Embora nos esforcemos para garantir a precisão, esteja ciente de que traduções automatizadas podem conter erros ou imprecisões. O documento original em seu idioma nativo deve ser considerado a fonte oficial. Para informações críticas, recomenda-se a tradução profissional realizada por humanos. Não nos responsabilizamos por quaisquer mal-entendidos ou interpretações equivocadas decorrentes do uso desta tradução.
